# Chapter 3 Finding Null Space

In [122]:
# numerical and scientific computing libraries  
import numpy as np 
import scipy as sp

# plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [123]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

In [124]:
# create a random matrix of size m x n with the rank <= k <= min(m, n).
def create_random_matrix(m: int, n: int, k: int) -> np.ndarray:
    if k > min(m, n):
        raise ValueError("k must be less than or equal to min(n, m)")
    A = np.random.randn(m, k)
    B = np.random.randn(k, n)
    return A@B

We consider a matrix of size $10 \times 11$ of rank 7, which is non-square and not of full rank.

In [125]:
m=10
n=11
k=7
# generate an mxn matrix of rank at most k
A = create_random_matrix(m, n, k)

To find a null space of a matrix $A$, we apply $LU$-decomposition to $A$. Since the null spaces of $A$ and $U$ coincide, we find a null space of the upper triangular matrix $U$. We characterize the pivot entries by searching the first non-zero element in each row of $U$, and then remaining variables are free ones.

In [ ]:
pivots = []
# LU-decomposition
P, L, U = sp.linalg.lu(A)
print(U)
# identify rank of U
rank_u = np.linalg.matrix_rank(U)
# find pivot entries
for i in range(rank_u):
    non_zero_indices = np.nonzero(U[i,:])
    pivots.append(non_zero_indices[0][0])
print('Pivot variables =')
print(np.int64(pivots))
# find column indices of free variables
free_variables = np.sort(list(set(range(n))- set(pivots)))
print("Free variables =")
print(free_variables)

[[-6.6486  0.2688  2.0031 -0.0759 -0.681   0.7743  2.0962  1.1056 -1.5248 -0.5511 -2.4102]
 [ 0.      5.0315  3.7506  5.3958  3.382  -1.8644 -2.6579 -3.6646 -5.5279 -0.2644  2.4499]
 [ 0.      0.      3.3443  0.9225 -1.0476 -0.2602 -2.2049 -0.877  -0.7328  1.2869 -1.414 ]
 [ 0.      0.      0.      5.9176  1.5869 -1.3519 -5.5978  1.9608 -3.276  -3.3937  3.0175]
 [ 0.      0.      0.      0.     -1.641  -0.6297  2.4833  3.9012 -0.3683 -6.875   1.5915]
 [ 0.      0.      0.      0.      0.     -1.0856 -4.0689 -3.5509  1.8101  4.3999 -0.0794]
 [ 0.      0.      0.      0.      0.      0.     -2.0918 -2.3091 -0.0107  3.7088 -0.5242]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.     -0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.     -0.     -0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.      0.     -0.    ]]
Pivot variables =
[0 1 2 3 4 5 6]
Free variables =
[ 7  8  9 10]


Note that we invoke **numpy.linalg.matrix_rank** which might use advanced results not covered until Chapter 3 in our textbook. We will implement this functionality from the scratch in a file 'Solving Non-invertible Linear System'. 

Collect pivot columns of U into **U_pivot**, and columns for free variables into **U_free**. Then solve $U_{pivot} X = -U_{free}$ where $X$ is an $n \times (n- Rank A)$ matrix solution of the equation.

In [127]:
# Collect pivot columns and free variables
U_pivot = U[:rank_u, pivots]
print('pivot columns of U = ')
print(U_pivot)
U_free = U[:rank_u, free_variables]
print('free variable columns of U = ')
print(U_free)
# Solve for the pivot part of a basis of the null space of A
X_pivot = np.linalg.solve(U_pivot,-U_free)
print('pivot part of X = ')
print(X_pivot)
# Combine pivot part and free variable part
X = np.eye(n)[:,free_variables]
X[pivots,:] = X_pivot

print('X = ')
print(X)

np.allclose(A@X, 0)

pivot columns of U = 
[[-6.6486  0.2688  2.0031 -0.0759 -0.681   0.7743  2.0962]
 [ 0.      5.0315  3.7506  5.3958  3.382  -1.8644 -2.6579]
 [ 0.      0.      3.3443  0.9225 -1.0476 -0.2602 -2.2049]
 [ 0.      0.      0.      5.9176  1.5869 -1.3519 -5.5978]
 [ 0.      0.      0.      0.     -1.641  -0.6297  2.4833]
 [ 0.      0.      0.      0.      0.     -1.0856 -4.0689]
 [ 0.      0.      0.      0.      0.      0.     -2.0918]]
free variable columns of U = 
[[ 1.1056 -1.5248 -0.5511 -2.4102]
 [-3.6646 -5.5279 -0.2644  2.4499]
 [-0.877  -0.7328  1.2869 -1.414 ]
 [ 1.9608 -3.276  -3.3937  3.0175]
 [ 3.9012 -0.3683 -6.875   1.5915]
 [-3.5509  1.8101  4.3999 -0.0794]
 [-2.3091 -0.0107  3.7088 -0.5242]]
pivot part of X = 
[[-0.0211  0.0168  0.1235 -0.196 ]
 [ 1.5317  1.2446 -1.4986 -0.2386]
 [ 0.0717 -0.2512 -0.0732  0.5765]
 [-1.278   1.1699  1.7957 -0.6184]
 [ 0.3744 -0.8794 -0.5117  0.2583]
 [ 0.8665  1.6865 -2.5923  0.866 ]
 [-1.1039 -0.0051  1.773  -0.2506]]
X = 
[[-0.0211  0.0168 

True

Check the linear independence of columns of matrix $X$ since, after row shuffling, a submatrix of $X$ is an identity matrix.